**Diana Kim ADA-2403M**

# **Assignment 3**
**Тема:** Архитектура GPU и оптимизация CUDA-программ


In [1]:
!nvidia-smi

Fri Jan 16 12:37:34 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


### **Задание 1**

Реализуйте программу на CUDA для поэлементной обработки массива (например,
умножение каждого элемента на число). Реализуйте две версии программы:
1. с использованием только глобальной памяти;
2. с использованием разделяемой памяти.
Сравните время выполнения обеих реализаций для массива размером 1 000 000
элементов.





In [9]:
%%writefile task1.cu

#include <cuda_runtime.h>                // подключает CUDA runtime
#include <iostream>                  // подключает вывод в консоль
#include <cstdlib>                       // подключает exit()
using namespace std;                               // чтобы не писать std::



void cuda_ok(cudaError_t err, const char* msg) {                // функция проверки ошибок CUDA
    if (err != cudaSuccess) {                                // если произошла ошибка
        cout << "CUDA error (" << msg << "): "                        // печатает место ошибки
             << cudaGetErrorString(err) << endl;                       // печатает текст ошибки
        exit(1);                                       // завершает программу
    }
}

__global__ void kernel_global(int* a, int n, int k) {                     // ядро с global memory
    int i = threadIdx.x + blockIdx.x * blockDim.x;                     // индекс элемента
    if (i < n) {                                           // если индекс в пределах массива
        a[i] = a[i] * k;                                // умножает элемент на k
    }
}
__global__ void kernel_shared(int* a, int n, int k) {                 // ядро с shared memory
    __shared__ int s_data[256];                                 // shared память на блок (под blockDim 256)
    int tid = threadIdx.x;                                         // локальный id потока в блоке
    int i = tid + blockIdx.x * blockDim.x;                       // глобальный индекс элемента

    if (i < n) {                            // если индекс в пределах массива
        s_data[tid] = a[i];                          // копирует элемент из global в shared
    } else {
        s_data[tid] = 0;                         // иначе кладёт 0, чтобы не было мусора
    }

    __syncthreads();                // ждет, пока все потоки загрузят shared
    s_data[tid] = s_data[tid] * k;            // умножает значение в shared
    __syncthreads();                                 // ждет, пока все потоки завершат умножение

    if (i < n) {                                       // если индекс в пределах массива
        a[i] = s_data[tid];                         // записывает результат обратно в global
    }
}

float run_kernel_global(int* d_a, int n, int k, int blockSize) {                  // функция запуска global kernel и замера времени
    int gridSize = (n + blockSize - 1) / blockSize;                        // считает количество блоков
    cudaEvent_t start, stop;                                 // создает события CUDA
    cuda_ok(cudaEventCreate(&start), "event create start");              // создает событие start
    cuda_ok(cudaEventCreate(&stop), "event create stop");                 // создает событие stop
    cuda_ok(cudaEventRecord(start), "event record start");                // старт замера времени
    kernel_global<<<gridSize, blockSize>>>(d_a, n, k);                      // запускает ядро
    cuda_ok(cudaGetLastError(), "kernel_global launch");                // проверяет запуск ядра
    cuda_ok(cudaEventRecord(stop), "event record stop");               // стоп замера времени
    cuda_ok(cudaEventSynchronize(stop), "event sync stop");               // ждет завершения
    float ms = 0.0f;                                                    // переменная для миллисекунд
    cuda_ok(cudaEventElapsedTime(&ms, start, stop), "elapsed time");            // считает время
    cuda_ok(cudaEventDestroy(start), "event destroy start");              // удаляет событие start
    cuda_ok(cudaEventDestroy(stop), "event destroy stop");                        // удаляет событие stop
    return ms;                          // возвращает время
}

float run_kernel_shared(int* d_a, int n, int k, int blockSize) {                  // функция запуска shared kernel и замера времени
    int gridSize = (n + blockSize - 1) / blockSize;                              // считает количество блоков
    cudaEvent_t start, stop;                                          // создает события CUDA
    cuda_ok(cudaEventCreate(&start), "event create start");                  // создает событие start
    cuda_ok(cudaEventCreate(&stop), "event create stop");                 // создает событие stop
    cuda_ok(cudaEventRecord(start), "event record start");                 // старт замера времени
    kernel_shared<<<gridSize, blockSize>>>(d_a, n, k);                    // запускает ядро
    cuda_ok(cudaGetLastError(), "kernel_shared launch");                   // проверяет запуск ядра
    cuda_ok(cudaEventRecord(stop), "event record stop");                     // стоп замера времени
    cuda_ok(cudaEventSynchronize(stop), "event sync stop");                     // ждет завершения
    float ms = 0.0f;                                                   // переменная для миллисекунд
    cuda_ok(cudaEventElapsedTime(&ms, start, stop), "elapsed time");                 // считает время
    cuda_ok(cudaEventDestroy(start), "event destroy start");                         // удаляет событие start
    cuda_ok(cudaEventDestroy(stop), "event destroy stop");                             // удаляет событие stop
    return ms;                                        // возвращает время
}

bool check_result(int* a, int n, int k) {                   // проверка корректности результата
    for (int i = 0; i < n; i++) {                      // проходит по массиву
        int expected = i * k;                           // ожидаемое значение
        if (a[i] != expected) {                        // если значение не совпало
            return false;                                  // возвращает false
        }
    }
    return true;                                    // если все совпало
}



int main() {
    const int N = 1000000;                     // размер массива
    const int K = 2;                               // множитель
    const int BLOCK = 256;                 // размер блока
    int* h_a = new int[N];                    // массив на CPU

    for (int i = 0; i < N; i++) {                   // заполняет массив
        h_a[i] = i;                            // кладет значение i
    }

    int* d_a = nullptr;                        // указатель на массив на GPU
    cuda_ok(cudaMalloc((void**)&d_a, N * (int)sizeof(int)), "cudaMalloc d_a");                     // выделяет память на GPU
    cuda_ok(cudaMemcpy(d_a, h_a, N * (int)sizeof(int), cudaMemcpyHostToDevice), "copy a to GPU");                      // копирует на GPU
    float t_global = run_kernel_global(d_a, N, K, BLOCK);                  // запускает global и меряет время
    cuda_ok(cudaMemcpy(h_a, d_a, N * (int)sizeof(int), cudaMemcpyDeviceToHost), "copy a to CPU");               // копирует на CPU
    bool ok_global = check_result(h_a, N, K);                                // проверяет корректность global

    for (int i = 0; i < N; i++) {                  // возвращает исходные данные
        h_a[i] = i;                           // снова i
    }
    cuda_ok(cudaMemcpy(d_a, h_a, N * (int)sizeof(int), cudaMemcpyHostToDevice), "copy a to GPU again");                 // снова на GPU
    float t_shared = run_kernel_shared(d_a, N, K, BLOCK);              // запускает shared и меряет время
    cuda_ok(cudaMemcpy(h_a, d_a, N * (int)sizeof(int), cudaMemcpyDeviceToHost), "copy a to CPU again");                        // копирует на CPU
    bool ok_shared = check_result(h_a, N, K);                            // проверяет корректность shared

    cout << "N = " << N << ", K = " << K << endl;                          // печатает параметры
    cout << "global time: " << t_global << endl;                     // печатает время global
    cout << "shared time: " << t_shared << endl;                         // печатает время shared
    cout << "check global: " << (ok_global ? "ok" : "fail") << endl;                            // печатает проверку global
    cout << "check shared: " << (ok_shared ? "ok" : "fail") << endl;                         // печатает проверку shared

    cuda_ok(cudaFree(d_a), "cudaFree d_a");                // освобождает память на GPU
    delete[] h_a;                                            // освобождает память на CPU

    return 0;               // завершает программу
}

Overwriting task1.cu


In [10]:
!nvcc task1.cu -o task1 -arch=compute_75 -code=sm_75

In [11]:
!./task1

N = 1000000, K = 2
global time: 0.079776
shared time: 0.04944
check global: ok
check shared: ok


### **Задание 2**

Реализуйте CUDA-программу для поэлементного сложения двух массивов. Исследуйте
влияние размера блока потоков на производительность программы. Проведите замеры
времени для как минимум трёх различных размеров блока.

In [19]:
%%writefile task2.cu

#include <cuda_runtime.h>                // подключает CUDA runtime
#include <iostream>                  // подключает вывод в консоль
#include <cstdlib>                       // подключает exit()
using namespace std;                               // чтобы не писать std::


void cuda_ok(cudaError_t err, const char* msg) {     // функция проверки ошибок CUDA
    if (err != cudaSuccess) {     // если произошла ошибка
        cout << "CUDA error (" << msg << "): ";     // печатает место ошибки
        cout << cudaGetErrorString(err) << endl;     // печатает текст ошибки
        exit(1);     // завершает программу
    }
}

__global__ void kernel_add(const int* a, const int* b, int* c, int n) {      // CUDA-ядро сложения
    int i = threadIdx.x + blockIdx.x * blockDim.x;      // вычисляет глобальный индекс
    if (i < n) {      // проверяет границы массива
        c[i] = a[i] + b[i];      // складывает элементы
    }
}

float run_add(const int* d_a, const int* d_b, int* d_c, int n, int blockSize) {       // функция запуска ядра и замера времени
    int gridSize = (n + blockSize - 1) / blockSize;       // считает количество блоков

    cudaEvent_t start, stop;       // создает события CUDA
    cuda_ok(cudaEventCreate(&start), "event create start");      // создает start
    cuda_ok(cudaEventCreate(&stop), "event create stop");      // создает stop

    cuda_ok(cudaEventRecord(start), "event record start");       // старт таймера
    kernel_add<<<gridSize, blockSize>>>(d_a, d_b, d_c, n);         // запускает ядро
    cuda_ok(cudaGetLastError(), "kernel_add launch");        // проверяет запуск ядра
    cuda_ok(cudaEventRecord(stop), "event record stop");            // стоп таймера
    cuda_ok(cudaEventSynchronize(stop), "event sync stop");              // ждет завершения ядра

    float ms = 0.0f;                  // переменная для времени
    cuda_ok(cudaEventElapsedTime(&ms, start, stop), "elapsed time");            // получает время в мс

    cuda_ok(cudaEventDestroy(start), "event destroy start");            // удаляет событие start
    cuda_ok(cudaEventDestroy(stop), "event destroy stop");                    // удаляет событие stop

    return ms;                    // возвращает время
}

bool check_sum(const int* a, const int* b, const int* c, int n) {              // проверка корректности
    for (int i = 0; i < n; i++) {                      // проходит по всем элементам
        int expected = a[i] + b[i];                            // считает ожидаемое значение
        if (c[i] != expected) {                     // если значение не совпало
            return false;                        // возвращает false
        }
    }
    return true;                 // если все правильно
}




int main() {
    const int N = 1000000;               // размер массивов
    const int BLOCKS[3] = {128, 256, 512};                 // три размера блока для теста
    int* h_a = new int[N];            // массив a на CPU
    int* h_b = new int[N];            // массив b на CPU
    int* h_c = new int[N];                     // массив c на CPU
    for (int i = 0; i < N; i++) {                  // заполняет массивы
        h_a[i] = i; // a[i] = i
        h_b[i] = 2 * i; // b[i] = 2*i
        h_c[i] = 0; // c[i] = 0
    }
    int* d_a = nullptr;             // указатель a на GPU
    int* d_b = nullptr;                   // указатель b на GPU
    int* d_c = nullptr;                        // указатель c на GPU

    cuda_ok(cudaMalloc((void**)&d_a, N * (int)sizeof(int)), "cudaMalloc d_a");        // выделяет память под a
    cuda_ok(cudaMalloc((void**)&d_b, N * (int)sizeof(int)), "cudaMalloc d_b");                        // выделяет память под b
    cuda_ok(cudaMalloc((void**)&d_c, N * (int)sizeof(int)), "cudaMalloc d_c");               // выделяет память под c
    cuda_ok(cudaMemcpy(d_a, h_a, N * (int)sizeof(int), cudaMemcpyHostToDevice), "copy a to GPU");           // копирует a на GPU
    cuda_ok(cudaMemcpy(d_b, h_b, N * (int)sizeof(int), cudaMemcpyHostToDevice), "copy b to GPU");                 // копирует b на GPU

    cout << "N = " << N << endl;               // печатает размер

    for (int t = 0; t < 3; t++) {              // цикл по трем размерам блока
        int blockSize = BLOCKS[t];                            // берет текущий block size
        cuda_ok(cudaMemset(d_c, 0, N * (int)sizeof(int)), "memset d_c");               // обнуляет c на GPU
        float ms = run_add(d_a, d_b, d_c, N, blockSize);                // запускает ядро и меряет время
        cuda_ok(cudaMemcpy(h_c, d_c, N * (int)sizeof(int), cudaMemcpyDeviceToHost), "copy c to CPU");           // копирует результат на CPU
        bool ok = check_sum(h_a, h_b, h_c, N);             // проверяет корректность
        cout << "block size = " << blockSize;                        // печатает block size
        cout << " time = " << ms;                  // печатает время
        cout << " check = " << (ok ? "ok" : "fail") << endl;        // печатает результат проверки
    }

    cuda_ok(cudaFree(d_a), "cudaFree d_a");                  // освобождает память a на GPU
    cuda_ok(cudaFree(d_b), "cudaFree d_b");                    // освобождает память b на GPU
    cuda_ok(cudaFree(d_c), "cudaFree d_c");                          // освобождает память c на GPU
    delete[] h_a;          // освобождает a на CPU
    delete[] h_b;                // освобождает b на CPU
    delete[] h_c;                   // освобождает c на CPU

    return 0;                          // завершает программу
}

Overwriting task2.cu


In [20]:
!nvcc task2.cu -o task2 -arch=compute_75 -code=sm_75

In [21]:
!./task2

N = 1000000
block size = 128 time = 0.173888 check = ok
block size = 256 time = 0.053408 check = ok
block size = 512 time = 0.053408 check = ok


### **Задание 3**

Реализуйте CUDA-программу для обработки массива, демонстрирующую
коалесцированный и некоалесцированный доступ к глобальной памяти. Сравните время
выполнения обеих реализаций для массива размером 1 000 000 элементов.

In [28]:
%%writefile task3.cu

#include <cuda_runtime.h>                 // подключаем CUDA runtime
#include <iostream>                             // для вывода
#include <cstdlib>                       // для exit()
using namespace std;                         // чтобы не писать std::


void cuda_ok(cudaError_t err, const char* msg) {              // объявляет функцию проверки ошибок CUDA
    if (err != cudaSuccess) {                       // проверяет наличие ошибки
        cout << "CUDA error (" << msg << "): ";            // выводит место ошибки
        cout << cudaGetErrorString(err) << endl;               // выводит текст ошибки
        exit(1);              // завершает программу
    }
}

__global__ void kernel_coalesced(int* a, int n) {           // объявляет ядро с коалесцированным доступом
    int i = threadIdx.x + blockIdx.x * blockDim.x;                       // вычисляет глобальный индекс
    if (i < n) {                  // проверяет выход за границы массива
        a[i] = a[i] + 1;                    // увеличивает элемент на 1
    }
}

__global__ void kernel_noncoalesced(int* a, int n, int stride) {              // объявляет ядро с некоалесцированным доступом
    int i = threadIdx.x + blockIdx.x * blockDim.x;                            // вычисляет глобальный индекс
    if (i < n) {                // проверяет границы массива
        int idx = (i * stride) % n;                // вычисляет индекс со stride
        a[idx] = a[idx] + 1;             // увеличивает элемент по скачущему индексу
    }
}

float run_coalesced(int* d_a, int n, int blockSize) {                   // объявляет функцию запуска coalesced ядра
    int gridSize = (n + blockSize - 1) / blockSize;                // вычисляет количество блоков
    cudaEvent_t start, stop;                 // объявляет CUDA-события
    cuda_ok(cudaEventCreate(&start), "event create start");               // создает start
    cuda_ok(cudaEventCreate(&stop), "event create stop");            // создаёт stop
    cuda_ok(cudaEventRecord(start), "event record start");       // запускает таймер
    kernel_coalesced<<<gridSize, blockSize>>>(d_a, n);                  // запускает ядро
    cuda_ok(cudaGetLastError(), "kernel_coalesced launch");                         // проверяет запуск ядра
    cuda_ok(cudaEventRecord(stop), "event record stop");                     // останавливает таймер
    cuda_ok(cudaEventSynchronize(stop), "event sync stop");            // ожидает завершения ядра
    float ms = 0.0f;             // объявляет переменную для времени
    cuda_ok(cudaEventElapsedTime(&ms, start, stop), "elapsed time");                 // получает время в мс
    cuda_ok(cudaEventDestroy(start), "event destroy start");        // удаляет start
    cuda_ok(cudaEventDestroy(stop), "event destroy stop");                             // удаляет stop
    return ms;            // возвращает время
}

float run_noncoalesced(int* d_a, int n, int blockSize, int stride) {                     // объявляет функцию запуска noncoalesced ядра
    int gridSize = (n + blockSize - 1) / blockSize;                              // вычисляет количество блоков
    cudaEvent_t start, stop;                    // объявляет CUDA-события
    cuda_ok(cudaEventCreate(&start), "event create start");                      // создает start
    cuda_ok(cudaEventCreate(&stop), "event create stop");               // создает stop
    cuda_ok(cudaEventRecord(start), "event record start");                // запускает таймер
    kernel_noncoalesced<<<gridSize, blockSize>>>(d_a, n, stride);                    // запускает ядро
    cuda_ok(cudaGetLastError(), "kernel_noncoalesced launch");               // проверяет запуск ядра
    cuda_ok(cudaEventRecord(stop), "event record stop");                        // останавливает таймер
    cuda_ok(cudaEventSynchronize(stop), "event sync stop");                  // ожидает завершения ядра
    float ms = 0.0f;                                         // объявляет переменную для времени
    cuda_ok(cudaEventElapsedTime(&ms, start, stop), "elapsed time");                       // получает время в мс
    cuda_ok(cudaEventDestroy(start), "event destroy start");      // удаляет start
    cuda_ok(cudaEventDestroy(stop), "event destroy stop");                  // удаляет stop
    return ms;          // возвращает время
}

bool check_plus_one(const int* a, int n) {                // объявляет функцию проверки результата
    for (int i = 0; i < n; i++) {               // проходит по массиву
        if (a[i] != i + 1) {                  // проверяет ожидаемое значение
            return false;                           // возвращает false при ошибке
        }
    }
    return true;             // возвращает true если всё правильно
}




int main() {
    const int N = 1000000;          // задает размер массива
    const int BLOCK = 256;                // задает размер блока
    const int STRIDE = 33;                // задает stride для плохого доступа
    int* h_a = new int[N];                    // выделяет массив на CPU
    for (int i = 0; i < N; i++) {               // заполняет массив
        h_a[i] = i;                   // записывает i
    }
    int* d_a = nullptr;                     // объявляет указатель на GPU
    cuda_ok(cudaMalloc((void**)&d_a, N * (int)sizeof(int)), "cudaMalloc d_a");          // выделяет память на GPU
    cuda_ok(cudaMemcpy(d_a, h_a, N * (int)sizeof(int), cudaMemcpyHostToDevice), "copy to GPU");               // копирует на GPU
    float t_coal = run_coalesced(d_a, N, BLOCK);                  // запускает coalesced и измеряет время
    cuda_ok(cudaMemcpy(h_a, d_a, N * (int)sizeof(int), cudaMemcpyDeviceToHost), "copy back coal");              // копирует результат
    bool ok_coal = check_plus_one(h_a, N);                            // проверяет результат coalesced
    for (int i = 0; i < N; i++) {                         // восстанавливает исходные значения
        h_a[i] = i;                           // снова записывает i
    }
    cuda_ok(cudaMemcpy(d_a, h_a, N * (int)sizeof(int), cudaMemcpyHostToDevice), "copy to GPU again");                 // копирует снова
    float t_non = run_noncoalesced(d_a, N, BLOCK, STRIDE);                            // запускает noncoalesced и измеряет время
    cuda_ok(cudaMemcpy(h_a, d_a, N * (int)sizeof(int), cudaMemcpyDeviceToHost), "copy back non");                // копирует результат
    bool ok_non = check_plus_one(h_a, N);                                       // проверяет результат noncoalesced

    cout << "N = " << N << endl;                // выводит N
    cout << "block = " << BLOCK << endl;            // выводит block
    cout << "stride = " << STRIDE << endl;              // выводит stride
    cout << "coalesced time: " << t_coal << endl;              // выводит время coalesced
    cout << "non-coalesced time: " << t_non << endl;                  // выводит время noncoalesced
    cout << "check coalesced: " << (ok_coal ? "ok" : "fail") << endl;           // выводит проверку coalesced
    cout << "check non-coalesced: " << (ok_non ? "ok" : "fail") << endl;               // выводит проверку noncoalesced

    cuda_ok(cudaFree(d_a), "cudaFree d_a");             // освобождает GPU память
    delete[] h_a;                    // освобождает CPU память

    return 0;          // завершает программу
}

Overwriting task3.cu


In [29]:
!nvcc task3.cu -o task3 -arch=compute_75 -code=sm_75

In [30]:
!./task3

N = 1000000
block = 256
stride = 33
coalesced time: 0.08272
non-coalesced time: 0.24592
check coalesced: ok
check non-coalesced: ok


### **Задание 4**

Для одной из реализованных в предыдущих заданиях CUDA-программ подберите
оптимальные параметры конфигурации сетки и блоков потоков. Сравните
производительность неоптимальной и оптимизированной конфигураций.

In [34]:
%%writefile task4.cu

#include <cuda_runtime.h>               // подключает CUDA runtime
#include <iostream>                // подключает вывод в консоль
#include <cstdlib>                   // подключает exit()
using namespace std;                      // убирает необходимость писать std::


void cuda_ok(cudaError_t err, const char* msg) {                  // объявляет функцию проверки ошибок CUDA
    if (err != cudaSuccess) {                        // проверяет наличие ошибки
        cout << "CUDA error (" << msg << "): ";                 // выводит место ошибки
        cout << cudaGetErrorString(err) << endl;                    // выводит текст ошибки
        exit(1);                   // завершает программу
    }
}

__global__ void kernel_add(const int* a, const int* b, int* c, int n) {                 // объявляет ядро сложения
    int i = threadIdx.x + blockIdx.x * blockDim.x;                        // вычисляет глобальный индекс
    if (i < n) {                        // проверяет границы массива
        c[i] = a[i] + b[i];                   // записывает сумму
    }
}

float run_add(const int* d_a, const int* d_b, int* d_c, int n, int blockSize) {                 // объявляет функцию запуска и замера времени
    int gridSize = (n + blockSize - 1) / blockSize;                    // вычисляет количество блоков
    cudaEvent_t start, stop;                                  // объявляет CUDA-события
    cuda_ok(cudaEventCreate(&start), "event create start");                    // создаёт start
    cuda_ok(cudaEventCreate(&stop), "event create stop");                          // создаёт stop
    cuda_ok(cudaEventRecord(start), "event record start");                    // запускает таймер
    kernel_add<<<gridSize, blockSize>>>(d_a, d_b, d_c, n);                       // запускает ядро
    cuda_ok(cudaGetLastError(), "kernel_add launch");                    // проверяет запуск ядра
    cuda_ok(cudaEventRecord(stop), "event record stop");                  // останавливает таймер
    cuda_ok(cudaEventSynchronize(stop), "event sync stop");                       // ожидает завершения ядра
    float ms = 0.0f;                                                 // объявляет переменную времени
    cuda_ok(cudaEventElapsedTime(&ms, start, stop), "elapsed time");                   // получает время
    cuda_ok(cudaEventDestroy(start), "event destroy start");              // удаляет start
    cuda_ok(cudaEventDestroy(stop), "event destroy stop");                       // удаляет stop

    return ms;                  // возвращает время
}

bool check_sum(const int* a, const int* b, const int* c, int n) {                  // объявляет функцию проверки
    for (int i = 0; i < n; i++) {                         // проходит по массиву
        int expected = a[i] + b[i];                             // вычисляет ожидаемое значение
        if (c[i] != expected) {                                   // проверяет совпадение
            return false;                             // возвращает false при ошибке
        }
    }
    return true;                        // возвращает true при успехе
}



int main() {
    const int N = 1000000;        // задаёт размер массивов
    const int BLOCK_BAD = 32;               // задаёт неоптимальный block size
    const int BLOCK_GOOD = 256;        // задаёт более оптимальный block size
    int* h_a = new int[N];                    // выделяет массив a на CPU
    int* h_b = new int[N];               // выделяет массив b на CPU
    int* h_c = new int[N];                // выделяет массив c на CPU
    for (int i = 0; i < N; i++) {                 // заполняет массивы
        h_a[i] = i;                            // задаёт a[i]
        h_b[i] = 2 * i;                             // задаёт b[i]
        h_c[i] = 0;                            // обнуляет c[i]
    }
    int* d_a = nullptr;         // объявляет указатель a на GPU
    int* d_b = nullptr;                          // объявляет указатель b на GPU
    int* d_c = nullptr;                    // объявляет указатель c на GPU

    cuda_ok(cudaMalloc((void**)&d_a, N * (int)sizeof(int)), "cudaMalloc d_a");            // выделяет память под a
    cuda_ok(cudaMalloc((void**)&d_b, N * (int)sizeof(int)), "cudaMalloc d_b");                  // выделяет память под b
    cuda_ok(cudaMalloc((void**)&d_c, N * (int)sizeof(int)), "cudaMalloc d_c");                        // выделяет память под c
    cuda_ok(cudaMemcpy(d_a, h_a, N * (int)sizeof(int), cudaMemcpyHostToDevice), "copy a to GPU");          // копирует a на GPU
    cuda_ok(cudaMemcpy(d_b, h_b, N * (int)sizeof(int), cudaMemcpyHostToDevice), "copy b to GPU");              // копирует b на GPU
    cuda_ok(cudaMemset(d_c, 0, N * (int)sizeof(int)), "memset d_c");                // обнуляет c на GPU
    float t_bad = run_add(d_a, d_b, d_c, N, BLOCK_BAD);                      // измеряет время неоптимального варианта
    cuda_ok(cudaMemcpy(h_c, d_c, N * (int)sizeof(int), cudaMemcpyDeviceToHost), "copy c bad");                // копирует результат
    bool ok_bad = check_sum(h_a, h_b, h_c, N);                                   // проверяет корректность
    cuda_ok(cudaMemset(d_c, 0, N * (int)sizeof(int)), "memset d_c");                            // обнуляет c на GPU
    float t_good = run_add(d_a, d_b, d_c, N, BLOCK_GOOD);                      // измеряет время оптимального варианта
    cuda_ok(cudaMemcpy(h_c, d_c, N * (int)sizeof(int), cudaMemcpyDeviceToHost), "copy c good");                     // копирует результат
    bool ok_good = check_sum(h_a, h_b, h_c, N);                                        // проверяет корректность

    cout << "N = " << N << endl;                       // выводит N
    cout << "bad block size = " << BLOCK_BAD << endl;                        // выводит плохой block size
    cout << "good block size = " << BLOCK_GOOD << endl;                   // выводит хороший block size
    cout << "bad config time: " << t_bad << "check = " << (ok_bad ? "ok" : "fail") << endl;                  // выводит время и проверку
    cout << "good config time: " << t_good << "check = " << (ok_good ? "ok" : "fail") << endl;                  // выводит время и проверку

    cuda_ok(cudaFree(d_a), "cudaFree d_a");            // освобождает d_a
    cuda_ok(cudaFree(d_b), "cudaFree d_b");                // освобождает d_b
    cuda_ok(cudaFree(d_c), "cudaFree d_c");                           // освобождает d_c
    delete[] h_a;                     // освобождает h_a
    delete[] h_b;                            // освобождает h_b
    delete[] h_c;                             // освобождает h_c

    return 0;            // завершает программу
}

Overwriting task4.cu


In [35]:
!nvcc task4.cu -o task4 -arch=compute_75 -code=sm_75

In [36]:
!./task4

N = 1000000
bad block size = 32
good block size = 256
bad config time: 0.172096check = ok
good config time: 0.054784check = ok
